In [1]:
import os

In [2]:
os.chdir("..")

In [3]:
pwd


'c:\\Users\\DELL\\Documents\\End_to_End_machine_learning_project_with_mlflow'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str